In [278]:
from urllib.request import urlopen as uReq

In [279]:
from bs4 import BeautifulSoup as soup

In [280]:
my_url = 'https://www.bls.gov/eag/eag.tx_collegestation_msa.htm'

In [281]:
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

In [282]:
page_soup = soup(page_html, 'html.parser')

In [283]:
data_list = page_soup.find_all('span', class_ = 'datavalue') #finds all the numbers on the page
data_list = data_list[20:] #skip the first worthless values
# data_list

In [285]:
page_soup.find_all('span', class_ = 'datavalue')[31].text #when there is a p this marks the end of a line!!!

'(p)-0.9'

In [297]:
rows = page_soup.find_all('p', class_ = 'sub0') #every row label of table
row_text = []
for i,val in enumerate(rows):
    if '(3)' in rows[i].text:
        row_text.append(rows[i].text[:-3]) #remove pesky little 3
    else:
        row_text.append(rows[i].text)
row_text = row_text[2:] # create a list of strings that indicate rows
# row_text

In [298]:
titles_text = []
for i, val in enumerate(titles):
    titles_text.append(titles[i].text)
titles_text = titles_text[2:] #list of table columns will go in df
# titles_text


['Jan 2020', 'Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'June 2020']

In [299]:
# This part is tricky
#because half of our dataset is 12month change and therefore useless
#i've chose to skip every other row when adding data

row_dict = {text: [] for text in row_text} #initialize a dictionary with row titles as keys
n=0 # counts every row
w=0 #counts every evenly divisble row

for i,tag in enumerate(data_list):
#     print(n,w,i) #uncomment to see iteratables
    
    # if n is odd than row is not of interest hence continue allows loop to go to next data value(i)
    #however, if it is the end of the row indicated by (p) then we need to iterate w to indicate a new row
    if n %2 != 0: 
        
        
        
        if '(p)' in data_list[i].text:
            w+=1
            n+=1
        continue
        
    #if n is odd than row is of interest and we check if it is end of a line to figure out if we iterate n or not
    
    if '(p)' in data_list[i].text:# indicate when you've reached the end of a row!
        row_dict[row_text[w]].append(data_list[i].text[3:])# add to dict remove  (p) from text
        n+=1
    
    #else indicates that n is even(row is of interest) and it is not the last value of a row(don't iterate n or w)
    else:
        values.append(data_list[i].text)
        row_dict[row_text[w]].append(data_list[i].text)

In [300]:
row_dict #this is the dictionary, we'll turn it into a df

{'Total Nonfarm': ['123.4', '127.4', '127.0', '117.4', '118.6', '117.6'],
 'Mining, Logging, and Construction': ['7.7',
  '7.7',
  '7.7',
  '7.9',
  '8.3',
  '8.0'],
 'Manufacturing': ['6.3', '6.3', '6.3', '5.9', '6.2', '6.3'],
 'Trade, Transportation, and Utilities': ['17.0',
  '17.2',
  '16.9',
  '16.1',
  '16.0',
  '16.4'],
 'Information': ['1.4', '1.4', '1.4', '1.3', '1.3', '1.4'],
 'Financial Activities': ['3.9', '3.9', '3.9', '3.8', '4.1', '4.1'],
 'Professional and Business Services': ['9.9',
  '10.1',
  '10.1',
  '9.1',
  '9.2',
  '9.6'],
 'Education and Health Services': ['12.7',
  '13.0',
  '13.2',
  '11.8',
  '12.2',
  '12.6'],
 'Leisure and Hospitality': ['17.3', '18.1', '17.1', '14.5', '15.1', '15.4'],
 'Other Services': ['3.4', '3.4', '3.3', '2.7', '2.9', '3.1'],
 'Government': ['43.8', '46.3', '47.1', '44.3', '43.3', '40.7']}

In [301]:
import pandas as pd
pd.DataFrame.from_dict(row_dict, orient='index',columns = titles_text) #final data frame can now do some analysis


,Jan 2020,Feb 2020,Mar 2020,Apr 2020,May 2020,June 2020
Total Nonfarm,123.4,127.4,127.0,117.4,118.6,117.6
"Mining, Logging, and Construction",7.7,7.7,7.7,7.9,8.3,8.0
Manufacturing,6.3,6.3,6.3,5.9,6.2,6.3
"Trade, Transportation, and Utilities",17.0,17.2,16.9,16.1,16.0,16.4
Information,1.4,1.4,1.4,1.3,1.3,1.4
Financial Activities,3.9,3.9,3.9,3.8,4.1,4.1
Professional and Business Services,9.9,10.1,10.1,9.1,9.2,9.6
Education and Health Services,12.7,13.0,13.2,11.8,12.2,12.6
Leisure and Hospitality,17.3,18.1,17.1,14.5,15.1,15.4
Other Services,3.4,3.4,3.3,2.7,2.9,3.1
